In [41]:
sys.path.append("..")
import os
import utils
import plot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

In [142]:
dir_path = "../"
relative_path = "results/500_simulations_contact_based_2021_05_30_23_24_11"
folder_path = dir_path + relative_path
div_filepath = folder_path + "/div.csv"
S_filepath = folder_path + "/S.csv" 
I_filepath = folder_path + "/I.csv" 
new_infected_filepath = folder_path + "/new_infected.csv" 
new_deaths_filepath = folder_path + "/new_deaths.csv" 
vaccinated_filepath = folder_path + "/vaccinated.csv"
pop_age_info_filepath = folder_path + "/start_date_population_age_labels.pkl"

start_date, population, age_labels = utils.read_pickle(pop_age_info_filepath)
paths = utils.create_named_tuple('paths', 'filepaths.txt')
config = utils.create_named_tuple('config', paths.config)

div_df = pd.read_csv(div_filepath, index_col=0)
S_df = pd.read_csv(S_filepath, index_col=0)
I_df = pd.read_csv(I_filepath, index_col=0)
new_infected_df = pd.read_csv(new_infected_filepath, index_col=0)
new_deaths_df = pd.read_csv(new_deaths_filepath, index_col=0)
vaccinated_df = pd.read_csv(vaccinated_filepath, index_col=0)

nr_simulations = div_df.loc[len(div_df)-1].simulation_nr
nr_weeks = div_df.loc[len(div_df)-1].week_nr
nr_regions = len(population)
nr_age_groups = len(age_labels)

In [143]:
len(age_labels)

7

In [137]:
vaccines_available = np.zeros((nr_simulations, nr_weeks))
flow_scale = np.zeros((nr_simulations, nr_weeks))
contact_weights = np.zeros((nr_simulations, nr_weeks, 4))
for i in tqdm(range(nr_simulations)):
    vaccines_available[i, :] = div_df.loc[(i)*75:(i+1)*75 -1].to_numpy()[:,3]
    flow_scale[i, :] = div_df.loc[(i)*75:(i+1)*75-1].to_numpy()[:,4] 
    contact_weights[i, :, :] = div_df.loc[(i)*75:(i+1)*75-1].to_numpy()[:,5:]

100%|██████████| 500/500 [00:00<00:00, 1375.87it/s]


In [126]:
S_regions = np.zeros((nr_simulations, nr_weeks, nr_regions))
I_regions = np.zeros((nr_simulations, nr_weeks, nr_regions))
new_infected_regions = np.zeros((nr_simulations, nr_weeks, nr_regions))
new_deaths_regions = np.zeros((nr_simulations, nr_weeks, nr_regions))
vaccinated_regions = np.zeros((nr_simulations, nr_weeks, nr_regions))
S_age_groups = np.zeros((nr_simulations, nr_weeks, nr_age_groups))
I_age_groups = np.zeros((nr_simulations, nr_weeks, nr_age_groups))
new_infected_age_groups = np.zeros((nr_simulations, nr_weeks, nr_age_groups))
new_deaths_age_groups = np.zeros((nr_simulations, nr_weeks, nr_age_groups))
vaccinated_age_groups = np.zeros((nr_simulations, nr_weeks, nr_age_groups))

for i in tqdm(range(nr_simulations)):
    S_regions[i, :, :] = S_df.loc[(i)*75:(i+1)*75 - 1].to_numpy()[:,3:-7]
    S_age_groups[i, :, :] = S_df.loc[(i)*75:(i+1)*75 - 1].to_numpy()[:,-7:]
    I_regions[i, :, :] = I_df.loc[(i)*75:(i+1)*75 - 1].to_numpy()[:,3:-7]
    I_age_groups[i, :, :] = I_df.loc[(i)*75:(i+1)*75 - 1].to_numpy()[:,-7:]
    new_infected_regions[i, :, :] = new_infected_df.loc[(i)*75:(i+1)*75 - 1].to_numpy()[:,3:-7]
    new_infected_age_groups[i, :, :] = new_infected_df.loc[(i)*75:(i+1)*75 - 1].to_numpy()[:,-7:]
    new_deaths_regions[i, :, :] = new_deaths_df.loc[(i)*75:(i+1)*75 - 1].to_numpy()[:,3:-7]
    new_deaths_age_groups[i, :, :] = new_deaths_df.loc[(i)*75:(i+1)*75 - 1].to_numpy()[:,-7:]
    vaccinated_regions[i, :, :] = vaccinated_df.loc[(i)*75:(i+1)*75 - 1].to_numpy()[:,3:-7]
    vaccinated_age_groups[i, :, :] = vaccinated_df.loc[(i)*75:(i+1)*75 - 1].to_numpy()[:,-7:]

100%|██████████| 500/500 [00:09<00:00, 53.37it/s]
